In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [33]:
new=pd.read_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\TDP43\TDP-43 ready.xlsx",header=0, index_col="Gene names")
new.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,CTRL 9,CTRL 10,CTRL 11,CTRL 12,CTRL 13,CTRL 14,CTRL 15,CTRL 16,CTRL 17,CTRL 18
Gene names,,,,,,,,,,,,,,,,,,,,,
GFAP,3.651330e+08,3.633836e+08,4.753440e+07,6.593567e+07,1.283911e+08,4.347393e+08,3.892265e+08,6.511454e+08,1.439446e+08,9.106206e+07,...,7.290492e+07,7.556678e+07,3.588126e+08,3.896489e+07,3.167767e+07,6.471454e+07,4.586542e+07,1.654363e+07,2.793670e+08,4.315035e+07
PLEC,6.623384e+06,1.089608e+07,1.555829e+06,8.139110e+05,4.559969e+06,3.851810e+06,4.093942e+06,9.877101e+06,2.351922e+06,1.654222e+06,...,1.450615e+06,1.617956e+06,9.689353e+06,6.614572e+05,1.311037e+06,1.521537e+06,2.187329e+06,2.217620e+05,5.510473e+06,5.574513e+05
COL6A3,4.868477e+05,2.505235e+07,5.976021e+05,6.129189e+04,1.553524e+06,7.807293e+05,5.866031e+05,3.310299e+06,1.284539e+05,8.607358e+06,...,3.553641e+05,3.187415e+05,8.241949e+06,4.800173e+05,5.645007e+05,3.329288e+05,1.961503e+05,5.210892e+04,9.126360e+05,7.931478e+04
SPTAN1,2.682920e+06,6.526254e+06,3.414484e+05,6.231768e+04,1.268394e+06,2.056121e+06,1.250939e+06,2.754038e+06,7.663568e+05,6.286129e+05,...,7.623999e+05,5.242005e+05,4.005212e+06,5.472360e+05,6.310157e+05,1.354030e+06,6.051050e+05,4.576834e+05,3.483159e+06,2.944552e+05
FLNA,5.500479e+05,2.022427e+07,3.065685e+05,1.071699e+05,5.122231e+05,5.468162e+05,3.914106e+05,2.588361e+05,5.056046e+05,1.504808e+05,...,2.288799e+06,7.247057e+05,1.467594e+06,2.852784e+05,2.844814e+06,3.583525e+06,1.253763e+06,9.192313e+04,9.648530e+05,4.751561e+05


In [34]:
new.shape

(2752, 36)

In [35]:
#Replace 0 with NaN
new.replace(0, np.nan, inplace=True)


In [46]:
#Columns belonging to Group 1 and 2
ALS_columns = [col for col in new.columns if "ALS" in col]
Control_columns = [col for col in new.columns if "CTRL" in col]


# Filter rows where at least 50% of Group 1 values are present, do the same for Group 2
filtered_als = new[new[ALS_columns].notna().mean(axis=1) >= 0.5]
cont_filt = filtered_als[filtered_als[Control_columns].notna().mean(axis=1) >= 0.5]

In [47]:
cont_filt.shape

(771, 36)

In [71]:
# Automatically identify group columns by their prefixes
als = [col for col in cont_filt.columns if col.startswith('ALS ')]
control = [col for col in cont_filt.columns if col.startswith('CTRL ')]

# Perform t-tests row-wise
results = []
for i, row in cont_filt.iterrows():
    t_stat, p_value = ttest_ind(row[als].dropna(), row[control].dropna())
    results.append({'Gene names': i, 't_stat': t_stat, 'p_value': p_value})
    
# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Gene names,t_stat,p_value
0,GFAP,3.809276,0.000557
1,PLEC,1.971806,0.056812
2,COL6A3,1.311626,0.198434
3,SPTAN1,1.548854,0.130676
4,FLNA,0.483198,0.632053
...,...,...,...
766,PKN1,0.848241,0.406342
767,MYOZ3,2.121621,0.043959
768,HIP1,-0.911491,0.369818
769,CENPE,0.334002,0.741855


In [73]:
# Add the results DataFrame to the original data DataFrame
cont_filt = pd.concat([cont_filt, results_df.set_index('Gene names')], axis=1)
cont_filt.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,CTRL 11,CTRL 12,CTRL 13,CTRL 14,CTRL 15,CTRL 16,CTRL 17,CTRL 18,t_stat,p_value
Gene names,,,,,,,,,,,,,,,,,,,,,
GFAP,3.651330e+08,3.633836e+08,4.753440e+07,6.593567e+07,1.283911e+08,4.347393e+08,3.892265e+08,6.511454e+08,1.439446e+08,9.106206e+07,...,3.588126e+08,3.896489e+07,3.167767e+07,6.471454e+07,4.586542e+07,1.654363e+07,2.793670e+08,4.315035e+07,3.809276,0.000557
PLEC,6.623384e+06,1.089608e+07,1.555829e+06,8.139110e+05,4.559969e+06,3.851810e+06,4.093942e+06,9.877101e+06,2.351922e+06,1.654222e+06,...,9.689353e+06,6.614572e+05,1.311037e+06,1.521537e+06,2.187329e+06,2.217620e+05,5.510473e+06,5.574513e+05,1.971806,0.056812
COL6A3,4.868477e+05,2.505235e+07,5.976021e+05,6.129189e+04,1.553524e+06,7.807293e+05,5.866031e+05,3.310299e+06,1.284539e+05,8.607358e+06,...,8.241949e+06,4.800173e+05,5.645007e+05,3.329288e+05,1.961503e+05,5.210892e+04,9.126360e+05,7.931478e+04,1.311626,0.198434
SPTAN1,2.682920e+06,6.526254e+06,3.414484e+05,6.231768e+04,1.268394e+06,2.056121e+06,1.250939e+06,2.754038e+06,7.663568e+05,6.286129e+05,...,4.005212e+06,5.472360e+05,6.310157e+05,1.354030e+06,6.051050e+05,4.576834e+05,3.483159e+06,2.944552e+05,1.548854,0.130676
FLNA,5.500479e+05,2.022427e+07,3.065685e+05,1.071699e+05,5.122231e+05,5.468162e+05,3.914106e+05,2.588361e+05,5.056046e+05,1.504808e+05,...,1.467594e+06,2.852784e+05,2.844814e+06,3.583525e+06,1.253763e+06,9.192313e+04,9.648530e+05,4.751561e+05,0.483198,0.632053


In [49]:
cont_filt.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\TDP43\TDP-43 ready cleaned.xlsx")

In [50]:
# Set the new range
min_range = 1
max_range = 5

In [51]:
dis_min = cont_filt.min().min()
dis_max = cont_filt.max().max()
scaled_dis = (cont_filt - dis_min) / (dis_max - dis_min)
scaled_data = (scaled_dis * (max_range - min_range)) + min_range
scaled_data.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,CTRL 9,CTRL 10,CTRL 11,CTRL 12,CTRL 13,CTRL 14,CTRL 15,CTRL 16,CTRL 17,CTRL 18
Gene names,,,,,,,,,,,,,,,,,,,,,
GFAP,3.149372,3.139074,1.279811,1.388131,1.755778,3.559114,3.291200,4.833000,1.847334,1.536039,...,1.429156,1.444825,3.112167,1.229366,1.186469,1.380942,1.269986,1.097382,2.644506,1.254004
PLEC,1.038986,1.064137,1.009155,1.004788,1.026839,1.022671,1.024096,1.058139,1.013841,1.009734,...,1.008536,1.009521,1.057034,1.003890,1.007714,1.008953,1.012873,1.001302,1.032434,1.003278
COL6A3,1.002863,1.147469,1.003514,1.000357,1.009142,1.004592,1.003450,1.019483,1.000753,1.050664,...,1.002089,1.001873,1.048513,1.002822,1.003320,1.001956,1.001151,1.000303,1.005369,1.000464
SPTAN1,1.015790,1.038414,1.002007,1.000364,1.007463,1.012100,1.007360,1.016208,1.004508,1.003697,...,1.004485,1.003082,1.023574,1.003218,1.003711,1.007967,1.003559,1.002691,1.020500,1.001730
FLNA,1.003235,1.119048,1.001801,1.000628,1.003012,1.003216,1.002301,1.001520,1.002973,1.000882,...,1.013470,1.004263,1.008636,1.001676,1.016743,1.021091,1.007377,1.000538,1.005676,1.002794


In [52]:
scaled_data.to_excel (r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\TDP43\TDP-43 ready scaled.xlsx")

In [55]:
# Automatically identify group columns by their prefixes
als = [col for col in scaled_data.columns if col.startswith('ALS ')]
control = [col for col in scaled_data.columns if col.startswith('CTRL ')]

# Perform t-tests row-wise
results = []
for i, row in scaled_data.iterrows():
    t_stat, p_value = ttest_ind(row[als].dropna(), row[control].dropna())
    results.append({'Gene names': i, 't_stat': t_stat, 'p_value': p_value})
    
# Convert results to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Gene names,t_stat,p_value
0,GFAP,3.809276,0.000557
1,PLEC,1.971806,0.056812
2,COL6A3,1.311626,0.198434
3,SPTAN1,1.548854,0.130676
4,FLNA,0.483198,0.632053
...,...,...,...
766,PKN1,0.848241,0.406342
767,MYOZ3,2.121621,0.043959
768,HIP1,-0.911491,0.369818
769,CENPE,0.334002,0.741855


In [56]:
# Add the results DataFrame to the original data DataFrame
scaled_data = pd.concat([scaled_data, results_df.set_index('Gene names')], axis=1)
scaled_data.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,CTRL 11,CTRL 12,CTRL 13,CTRL 14,CTRL 15,CTRL 16,CTRL 17,CTRL 18,t_stat,p_value
Gene names,,,,,,,,,,,,,,,,,,,,,
GFAP,3.149372,3.139074,1.279811,1.388131,1.755778,3.559114,3.291200,4.833000,1.847334,1.536039,...,3.112167,1.229366,1.186469,1.380942,1.269986,1.097382,2.644506,1.254004,3.809276,0.000557
PLEC,1.038986,1.064137,1.009155,1.004788,1.026839,1.022671,1.024096,1.058139,1.013841,1.009734,...,1.057034,1.003890,1.007714,1.008953,1.012873,1.001302,1.032434,1.003278,1.971806,0.056812
COL6A3,1.002863,1.147469,1.003514,1.000357,1.009142,1.004592,1.003450,1.019483,1.000753,1.050664,...,1.048513,1.002822,1.003320,1.001956,1.001151,1.000303,1.005369,1.000464,1.311626,0.198434
SPTAN1,1.015790,1.038414,1.002007,1.000364,1.007463,1.012100,1.007360,1.016208,1.004508,1.003697,...,1.023574,1.003218,1.003711,1.007967,1.003559,1.002691,1.020500,1.001730,1.548854,0.130676
FLNA,1.003235,1.119048,1.001801,1.000628,1.003012,1.003216,1.002301,1.001520,1.002973,1.000882,...,1.008636,1.001676,1.016743,1.021091,1.007377,1.000538,1.005676,1.002794,0.483198,0.632053


In [57]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(scaled_data['p_value'], method='fdr_bh')
scaled_data['p_adjusted'] = p_adjusted
scaled_data.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,CTRL 12,CTRL 13,CTRL 14,CTRL 15,CTRL 16,CTRL 17,CTRL 18,t_stat,p_value,p_adjusted
Gene names,,,,,,,,,,,,,,,,,,,,,
GFAP,3.149372,3.139074,1.279811,1.388131,1.755778,3.559114,3.291200,4.833000,1.847334,1.536039,...,1.229366,1.186469,1.380942,1.269986,1.097382,2.644506,1.254004,3.809276,0.000557,0.214869
PLEC,1.038986,1.064137,1.009155,1.004788,1.026839,1.022671,1.024096,1.058139,1.013841,1.009734,...,1.003890,1.007714,1.008953,1.012873,1.001302,1.032434,1.003278,1.971806,0.056812,0.726282
COL6A3,1.002863,1.147469,1.003514,1.000357,1.009142,1.004592,1.003450,1.019483,1.000753,1.050664,...,1.002822,1.003320,1.001956,1.001151,1.000303,1.005369,1.000464,1.311626,0.198434,0.750563
SPTAN1,1.015790,1.038414,1.002007,1.000364,1.007463,1.012100,1.007360,1.016208,1.004508,1.003697,...,1.003218,1.003711,1.007967,1.003559,1.002691,1.020500,1.001730,1.548854,0.130676,0.750323
FLNA,1.003235,1.119048,1.001801,1.000628,1.003012,1.003216,1.002301,1.001520,1.002973,1.000882,...,1.001676,1.016743,1.021091,1.007377,1.000538,1.005676,1.002794,0.483198,0.632053,0.881216


In [58]:
_, p_adjusted, _, _ = multipletests(scaled_data['p_value'], method='bonferroni')
scaled_data['p_adjusted_bonf'] = p_adjusted
scaled_data.head()

,ALS 1,ALS 2,ALS 3,ALS 4,ALS 5,ALS 6,ALS 7,ALS 8,ALS 9,ALS 10,...,CTRL 13,CTRL 14,CTRL 15,CTRL 16,CTRL 17,CTRL 18,t_stat,p_value,p_adjusted,p_adjusted_bonf
Gene names,,,,,,,,,,,,,,,,,,,,,
GFAP,3.149372,3.139074,1.279811,1.388131,1.755778,3.559114,3.291200,4.833000,1.847334,1.536039,...,1.186469,1.380942,1.269986,1.097382,2.644506,1.254004,3.809276,0.000557,0.214869,0.429737
PLEC,1.038986,1.064137,1.009155,1.004788,1.026839,1.022671,1.024096,1.058139,1.013841,1.009734,...,1.007714,1.008953,1.012873,1.001302,1.032434,1.003278,1.971806,0.056812,0.726282,1.000000
COL6A3,1.002863,1.147469,1.003514,1.000357,1.009142,1.004592,1.003450,1.019483,1.000753,1.050664,...,1.003320,1.001956,1.001151,1.000303,1.005369,1.000464,1.311626,0.198434,0.750563,1.000000
SPTAN1,1.015790,1.038414,1.002007,1.000364,1.007463,1.012100,1.007360,1.016208,1.004508,1.003697,...,1.003711,1.007967,1.003559,1.002691,1.020500,1.001730,1.548854,0.130676,0.750323,1.000000
FLNA,1.003235,1.119048,1.001801,1.000628,1.003012,1.003216,1.002301,1.001520,1.002973,1.000882,...,1.016743,1.021091,1.007377,1.000538,1.005676,1.002794,0.483198,0.632053,0.881216,1.000000


In [62]:
#df['average'] = df.mean(numeric_only=True, axis=1)
scaled_data['Mean Control'] = scaled_data.loc[:, scaled_data.columns.str.startswith('CTRL ')].mean(axis=1)
scaled_data['Mean ALS'] = scaled_data.loc[:, scaled_data.columns.str.startswith('ALS ')].mean(axis=1)

In [63]:
#Fold Change
scaled_data["FC ALS-C"]= scaled_data["Mean ALS"]/scaled_data["Mean Control"]

In [64]:
#Log 2 FC
scaled_data["Log2FC ALS-C"]=np.log2(scaled_data["FC ALS-C"])

In [65]:
scaled_data.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\TDP43\TDP-43 scaled stats.xlsx")

In [66]:
# Sort the DataFrame by the absolute values of the column
down_sorted = scaled_data.loc[scaled_data["Log2FC ALS-C"].abs().sort_values(ascending=False).index]



In [67]:
down_sorted.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\TDP43\TDP-43 scaled sorted.xlsx")

In [69]:
# Filter rows where 'P-value' <= 0.05
filtered_down = down_sorted[down_sorted['p_value'] <= 0.05]

In [70]:
filtered_down.to_excel(r"C:\Users\Client\OneDrive - Queen Mary, University of London\PhD\PhD back up\Programing full\R\Datasets\TDP43\TDP-43 scaled filtered.xlsx")